In [7]:
import numpy as np 
import pandas as pd 
import os, gc, re, warnings
from transformers import AutoModel,AutoTokenizer
import torch
import torch.nn.functional as F
from tqdm import tqdm
import sys
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as nafc
from random import shuffle
from nlpaug.util import Action
warnings.filterwarnings("ignore")
# Set the following to avoid warning message
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [5]:
dftr = pd.read_csv("train.csv")
dftr["src"]="train"
dfte = pd.read_csv("test.csv")
dfte["src"]="test"
print('Train shape:',dftr.shape,'Test shape:',dfte.shape,'Test columns:',dfte.columns)
df = pd.concat([dftr,dfte],ignore_index=True)

dftr.head()

Train shape: (3911, 9) Test shape: (3, 3) Test columns: Index(['text_id', 'full_text', 'src'], dtype='object')


,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions,src
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0,train
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5,train
2,00299B378633,"Dear, Principal\n\nIf u change the school poli...",3.0,3.5,3.0,3.0,3.0,2.5,train
3,003885A45F42,The best time in life is when you become yours...,4.5,4.5,4.5,4.5,4.0,5.0,train
4,0049B1DF5CCC,Small act of kindness can impact in other peop...,2.5,3.0,3.0,3.0,2.5,2.5,train


In [16]:
def augment_text(df):
#     syn_aug = naw.SynonymAug(aug_src='wordnet')
#     back_trans_aug = naw.BackTranslationAug(device="cuda")
#     aug.aug_p=pr
    new_text=[]
    cohesion_new = []
    syntax_new = []
    vocabulary_new = []
    phraseology_new = []
    grammar_new = []
    conventions_new = []


    for i in tqdm(range(len(df))):
        text = df.iloc[i]['full_text']
#         aug_text = back_trans_aug.augment(text)[0]
#         syn_aug_text = syn_aug.augment(text)[0]
        
        word2vec_path = "GoogleNews-vectors-negative300.bin"
        w2v_aug = naw.WordEmbsAug(
            model_type='word2vec', model_path=word2vec_path,
            action="substitute")
        aug_text = w2v_aug.augment(text)[0]
        
        
        new_text.append(aug_text)
        cohesion_new.append(df.iloc[i]['cohesion'])
        syntax_new.append(df.iloc[i]['syntax'])
        vocabulary_new.append(df.iloc[i]['vocabulary'])
        phraseology_new.append(df.iloc[i]['phraseology'])
        grammar_new.append(df.iloc[i]['grammar'])
        conventions_new.append(df.iloc[i]['conventions'])
    
    ## dataframe
    new=pd.DataFrame({'full_text':new_text,'cohesion':cohesion_new,'syntax':syntax_new,'vocabulary':vocabulary_new,'phraseology':phraseology_new,'grammar':grammar_new,'conventions':conventions_new})

    del new_text, cohesion_new, syntax_new, vocabulary_new, phraseology_new, grammar_new, conventions_new
    gc.collect()

    return new

In [17]:
df_aug = augment_text(dftr)
df_aug["src"]="train"
df_aug.to_csv('w2v_aug.csv',index=None)

100%|████████████████████████████████████████████████████████████████████████████| 3911/3911 [3:09:44<00:00,  2.91s/it]


In [18]:
data = {'Name': ['Tom', 'nick', 'krish', 'jack'],
        'Age': [20, 21, 19, 18]}
a = pd.DataFrame(data)
a

,Name,Age
0,Tom,20
1,nick,21
2,krish,19
3,jack,18


In [22]:
a['new'] = np.array([[1,2,3],[],[],[]])

In [23]:
a

,Name,Age,new
0,Tom,20,"[1, 2, 3]"
1,nick,21,[]
2,krish,19,[]
3,jack,18,[]
